# Projet Archi BigData

## équipe
- Marwa Iben Khalifa
- Maxime Cordeiro
- Louis Poulin
- Ivan Braga Fernandes
- Hicham Hecharif Chefchaouni

## client
Binance

# contraintes
- serveur mongodb
- cluster Hadoop HDFS
- processing des données avec Apache Spark

# livrables demandés
- un support de présentation (ex: powerpoint)
- méthodologie
- présentation du sujet + source de donnée
- présentation de l’architecture (ex: draw io)
- justification des choix de technos
- difficultés rencontrées, axe d’amélioration

## Chargement de spark

In [39]:
from pyspark.sql import SparkSession
appName= "hive_pyspark"
master= "local"
spark = SparkSession.builder \
        .master(master).appName(appName).enableHiveSupport().getOrCreate()

In [40]:
path = "./data/BTCBUSD-4h-2022-11-22.csv"

## ajout du header

In [45]:
from pyspark.sql.types import *
schema = StructType() \
        .add("timestamp",StringType(),True) \
        .add("open",FloatType(),True) \
        .add("high",FloatType(),True) \
        .add("low",FloatType(),True) \
        .add("close",FloatType(),True) \
        .add("volume",FloatType(),True) \
        .add("close_time",StringType(),True) \
        .add("quote_asset_volume",FloatType(),True) \
        .add("number_of_trades",IntegerType(),True) \
        .add("taker_buy_base_asset_volume",FloatType(),True) \
        .add("taker_buy_quote_asset_volume",FloatType(),True) \
        .add("ignore",IntegerType(),True)
      
df_binance = spark.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load(path)
df_binance.show(5)

+-------------+--------+--------+--------+--------+---------+-------------+------------------+----------------+---------------------------+----------------------------+------+
|    timestamp|    open|    high|     low|   close|   volume|   close_time|quote_asset_volume|number_of_trades|taker_buy_base_asset_volume|taker_buy_quote_asset_volume|ignore|
+-------------+--------+--------+--------+--------+---------+-------------+------------------+----------------+---------------------------+----------------------------+------+
|1669075200000|15761.55|15955.62|15726.98|15828.25|20762.799|1669089599999|      3.28859616E8|          491909|                  10147.984|                  1.607348E8|     0|
|1669089600000|15828.28|15862.66| 15598.6|15742.88|20869.232|1669103999999|      3.28515104E8|          452242|                  10223.968|                 1.6093504E8|     0|
|1669104000000|15742.88|15805.61|15638.63| 15730.0|20420.682|1669118399999|       3.2082464E8|          451824|         

In [43]:
df_hive=spark.sql("show databases")
df_hive.show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [60]:
df_binance.write.saveAsTable("binance")

In [57]:
df1=spark.sql("select * from binance limit 5")
df1.show()

+-------------+--------+--------+--------+--------+---------+-------------+------------------+----------------+---------------------------+----------------------------+------+
|    timestamp|    open|    high|     low|   close|   volume|   close_time|quote_asset_volume|number_of_trades|taker_buy_base_asset_volume|taker_buy_quote_asset_volume|ignore|
+-------------+--------+--------+--------+--------+---------+-------------+------------------+----------------+---------------------------+----------------------------+------+
|1669075200000|15761.55|15955.62|15726.98|15828.25|20762.799|1669089599999|      3.28859616E8|          491909|                  10147.984|                  1.607348E8|     0|
|1669089600000|15828.28|15862.66| 15598.6|15742.88|20869.232|1669103999999|      3.28515104E8|          452242|                  10223.968|                 1.6093504E8|     0|
|1669104000000|15742.88|15805.61|15638.63| 15730.0|20420.682|1669118399999|       3.2082464E8|          451824|         